# Introducing Ts

namespace放在class的位置用：
namespace Foo{}

null，undefined在非严格模式下可以赋值给任何类型的variable
tsc -strictNullChecks file.ts

void类型是any类型的背面

在Ts中，tuple类型是形如:[string,number]的类型

undefined是指variable已经被声明而没有值的一种状态，null是variable指有值而值为null

spread Operation用在Array和object的create（merge，copy）上：
let copyObj = {...originalObj1}; 
let mergedArr = [...originalArr1, ...originalArr2];
let newObjObj = {... originalObj1, g: 7, h: 8};

variable或expression有value

enum AlertLevel和switch-case连用：
function getAlertSubscribers(level: AlertLevel) {
	let emails = new Array<string>();
	switch (level) {
		case AlertLevel.info:
			emails.push("cst@domain.com");
			break;

for...of可遍历Array，string，argumentsObject和map，set：
let iterable = [10, 20, 30];
for (let value of iterable) {
	console.log(value);
} 

# Chapter2 Working with Types

ts使用的是Structural type syste，也兼容Nominal type system。比如函数的parameter为Person类型，实际的argument①如果是new类传入的，这个类不需要写明implements Person，②如果是variable传入的，这个variable不需要写明:Person

typeGuards是将variable从 | 类型、any类型变为单类型的过程。

| 接口(Union types)是形如Supplier | Customer的接口，| 接口不能作为类的接口。

&接口(Intersection types)是形如Supplier & Customer的接口，&接口常作为类的接口

=====================================

试试tsc --noImplicitAny test.ts：
function fn(a,b){
	return a+b
}

type NumberType = typeof myNumber

=====================================

typeGuards总的有：
person:Supplier|Customer对应person.order != undefined，这个很烦，必须用isCustomer(person: ICustomer | ISupplier): person is ICustomer,称为自定义typeGuards。
person:Supplier|Customer对应person instanceof Supplier，这个简单
animal:IDog|ICat|Iman对应"woof" in animal，这个中等，虽然编译时判断:对象的woof、运行时判断:接口的woof，但起码都是值woof
value: number | number[] 对应Array.isArray(value)，这个简单。
shape:Cube | Pyramid | Sphere对应 shape.kind == "cube"，这个难。

自定义typeGuards：
interface ISupplier {
	order(): void;
}
interface ICustomer {
	buy(): void;
}
class Customer implements ICustomer {
	buy(): void { };
}

function isCustomer(person: ICustomer | ISupplier): person is ICustomer {
	return (<ICustomer>person).buy != undefined
}
function ffff(person: ICustomer | ISupplier) {
	if (isCustomer(person)) {
		person.buy()
		console.log("23333333")
	}
}
let myCustomer = new Customer()
ffff(myCustomer)

shape:Cube | Pyramid | Sphere对应 shape.kind == "cube"，这个难(前提是Cube接口中必须kind:"Cube”，实现Cube接口的类也必须kind:"Cube"=“Cube”。
☆☆☆☆☆☆类型的Cube是编译时给shape.kind == "Cube"做判断用的，值的Cube是运行时给shape.kind == "Cube"做判断用的)：
interface ICube{
	kind:"Cube"
	hhh():void;
}
interface IPyramid{
	kind:"Pyramid"
}

class Pyramid implements IPyramid{
	kind:"Pyramid" = "Pyramid"
}
class Cube implements ICube{
	kind:"Cube" = "Cube";
	hhh():void{}

}
function fn(shape:ICube|IPyramid){
	if(shape.kind =="Cube"){
		shape.hhh()
		console.log("OK")
	}
}
let myCube = new Cube()
fn(myCube)

=====================================

在编译时用typeof提取variable的类型，用keyof提取类型的属性名并转化成literalType的 | 类型：
let person = { name: "Remo", age: "28" };
interface User {
	name: string;
	age: number;
}
type userKeys = keyof typeof person; // "name" | "age" 


let a:Array<number|string> = ['1',2,3,4]
let b:string = a[0]

你可以在函数外定义接口，在函数内定义实现该接口的类。你可以在任何地方定义type，interface

=========================================

☆☆☆☆☆T接口有点类似于any类型。
定义function，interface时使用T接口，表明T接口暂时不定义。
定义function，interface时使用T extends IComparableWith<TT>，表明虽然T接口暂不定义，但是IComparableWith<TT>接口先给你用着：
interface IComparableWith<T> {
	isEquals(other: T): boolean;
}

function isEqual<T extends IComparableWith<T>>(one: T, another: T) {
	return one.isEquals(another)
}
function isEqual_<T extends IComparableWith<TT>, TT>(one: T, another: TT) {
	return one.isEquals(another)
}
比较：
interface T {
	isEquals(other: T): void;
}
function isEquals(one: T, another: T) {
	return one.isEquals(another)
}

☆☆☆☆☆接口的属性有属性类型和值类型，两个类型。属性名即属性的属性类型。
K extends keyof IUser表明K是IUser接口的属性类型以及属性名
IUser[K]指的是IUser中属性类型为K的类型：
function findObjectByValueInKey<T, K extends keyof T>(entities: T[], key: K, value: T[K]) {
	return entities.filter(entity => entity[key] == value)
}
比较：
interface IUser {
	id: number;
	name: string
}
function findUserByValueInKey<K extends keyof IUser>(entities: IUser[], key: K, value: IUser[K]) {
	return entities.filter(entity => entity[key] == value)
}
const users = [
        { name: "Smith", id: 28 },
        { name: "Johnson", id: 55 },
        { name: "Williams", id: 14 }
];
findUserByValueInKey<"name">(users,"name",'Smith')

===================================

immutable == readonly

type上出现T类型，要么是凭借他，要么是改变他。
//拿到一个列表类型里，元素的类型：
interface IUser {
	id: number;
	name: string
}
type Flatten_<T> = T extends Array<infer U> ? U :T;
type Flatten<T> = T extends Array<any> ? T[number] :T;//一个variable的类型决定另一个variable的类型
type IUserArray = IUser[][]
type number_ = Flatten<IUserArray>
//拿到一个函数类型里，返回值的类型：
type ReturnedType<T extends (...args:any[])=>any> = T extends (...args:any[])=>infer R? R :any;
type find = ()=>number
type b = ReturnedType<find>
//拿到 | 类型里，符合条件的类型
let aaa:Exclude<boolean|boolean[]|string[],any[]> = true
let nnn:ReturnType<func> = 3


# Chapter 3 Working with Functions

避免使用...args:any[]，请直接使用args:any[]

函数重载:
interface IDocument {
	createElement(tagName: "div"): HTMLDivElement;
	createElement(tagName: "span"): HTMLSpanElement;
	createElement(tagName: "canvas"): HTMLCanvasElement;
	//createElement(tagName: string): HTMLElement;
}
class Document_ implements IDocument {
	createElement(tagName: "div"): HTMLDivElement;
	createElement(tagName: "span"): HTMLSpanElement;
	createElement(tagName: "canvas"): HTMLCanvasElement;
	createElement(tagName: string): HTMLElement {
		return
	};
}

和：
function test(name: string): string;
function test(age: number): number;
function test(single: boolean): string;
function test(value: (string | number | boolean)): string | number {
	return
}

在大多数lexical-scoped语言中，变量是存活在{}块中的，而JavaScript中用var定义的变量是存活在函数中的。所以只有var变量存在var变量提升。
variable存活的区域，叫做scope。
变量提升，变量的值不提升。

虽然const变量不能二次赋值，但const变量不是immutable

===================================

异步操作==出内存的操作。
串行异步操作，自身阶段的操作在函数定义阶段定义，下步(成功和失败)操作在函数调用阶段定义。
在asyncFunction块之外才需要，.then().catch()

# Chapter 4 OOP with TypeScript

interface IRectangle_{
	width:string;
	height:string;
}
class Rectangle_ implements IRectangle_{
	width:string;
	height!:string;
	constructor(width:string){
		this.width = width
	}
}

let b:Readonly<number[]> = [1,2]

☆☆☆☆☆，类智能单继承，但接口可以多继承。 generic constraint 也是单的
interface FooBar extends Foo, Bar {}
class Example2<T extends FooBar> {

Association：无父子关系，生命周期各自独立。如老师与学生
Aggregation ：有父子关系，生命周期各自独立，如手机和手机电池。
Composition ：有父子关系，同一生命周期，如问题和答案。

applyMixins只涉及构造函数prototype中属性和方法的平移。
__ extends涉及构造函数①静态属性和方法的平移②prototype中属性和方法转移到对方prototype的__proto__中

============================

ES5及之前，对象的属性的key只能是字符串。ES6之后，对象的属性的key可以是字符串或symbol。当用variable的值作为对象属性的key时要加[]括号，所以用值为symbol的variable作为对象的属性时必须：
	obj = {
		[anySymbol]: "anyValue"
	};

Iterator是拥有next函数的对象，Iterable是拥有[Symbol.iterator]函数的对象，[Symbol.iterator]函数产生Iterator对象，IterableIterator即实现Iterable右实现了Iterator：
class Fib implements IterableIterator<number>{
	value = 0;
	nextValue = 1;
	next(): IteratorResult<number> {
		let current = this.value
		this.value = this.nextValue
		this.nextValue = current + this.value;
		return { done: false, value: current }
	}
	[Symbol.iterator](): IterableIterator<number> {
		return new Fib()
	}
}

=============================================

如果要将子类的函数名，在父类中也放一份，请将父类设置成abstract

在ts中，接口可继承类，多个类

=======================================


Email依赖Person：Email拥有validateEmail函数。Email输出email对象给Person。email验证好给我
Rectangle，Circle依赖AreaCalculator：Rectangle，Circle拥有area函数。Rectangle，Circle输出area函数给AreaCalculator。面积算好了给我。
CookiePersitanceService，LocalStoragePersitanceService依赖FavouritesController：前两者拥有save函数。前两者(dependency)输出save函数给后者(client)。我们这种将一个对象(dependency)的行为交由另一个对象(client)代理的行为，叫做dependency injection。☆☆☆☆☆在ts中，如果这种代理(injection)是间接的，也就是说，不是直接的类代理(inject)类，而是间接的类代理接口，则称这种代理接口的行为，称作dependency inversion

试试：
document.cookie = `${id}=${value}`;
function getUniqueId() {  
  return Math.random().toString(36).substr(2, 9); 
}

# Chapter5 Working with Dependencies

bower落伍了，yarn新潮

npm init --yes
npm install
npm outdated

在ts中CommonJS的语法任何情况都不会被转化。tsconfig.json中module:"amd"最保险，此时es2015语法会被转化amd，前端需要requireJS配合使用。

# Chapter6  Understanding the runtime

函数被bind后，则对apply和call失效。

构造函数构造对象。在ts中，构造函数外套一层IIFE

使用this.constructor在对象方法中call类属性，如：
this.constructor.PI

访问prototype只推荐Person.prototype

类的属性、方法==静态属性、静态方法，对所有对象的对象方法开放，也对全局开放。
☆☆☆☆☆类的方法被多个对象的方法共享的行为，这叫singleton行为。这个类方法，可以叫context。
☆☆☆☆☆闭包方法1，是①对全局私有②被对象方法共享的方法，类似于静态方法。附加闭包方法1的类，只需要在class外套一个IIFE：
let Counter_ = function() {
	let _COUNT = 0;
	class Counter{
		increase(){
			_COUNT +=1
			return _COUNT
		}
	}
	return Counter
}()
let counter1 = new Counter_()
console.log(counter1.increase())
let counter2 = new Counter_()
console.log(counter1.increase())

☆☆☆☆☆闭包方法2，是①对全局私有②对象之间也私有的方法，附加闭包方法2的类，call时不需要new：
let Counter = function() {
	let _COUNT = 0
	class Counter {
		increase() {
			_COUNT += 1
			return _COUNT
		}
	}
	return new Counter()
}
let counter1 = Counter()
console.log(counter1.increase())
let counter2 = Counter()
console.log(counter2.increase())

# Chapter 7 FP

OOP程序的构件是object，FP程序的构件是function

☆☆☆☆☆compose函数就像只有一个属性和只有一个方法的类

执行器==执行函数业务逻辑的函数

FP精髓是函数的嵌套，和函数的waterfall。

call compose函数来储存函数，compose函数返回值为执行器，call执行器会顺序执行被储存的函数。
let compose2: <T>(f: (x: T) => T, g: (x: T) => T) => (x: T) => T
let compose2_: <T1,T2,T3>(f: (x: T2) => T3, g: (x: T1) => T2 ) => (x: T1) => T3;
compose2_ = <T>(f, g) => (x: T) => f(g(x))
let compose3 = (f, g, h) => compose2(compose2(f, g), h)

终极函数储存器：
let composeX = (funcs: []) => {
	arg => funcs.reduce((prev, current:(arg:any)=>any) => current(prev), arg)
}

partial application，是指对一个函数的不完全使用，不执行主逻辑只储存参数，返回值为执行器。依据传参情况决定对函数是部分使用还是完全使用(complete application)

function curry2(fn){
	return a=>b=>fn(a,b)
}
function curry3(fn){
	return a=>b=>c=>fn(a,b,c)
}
☆☆☆☆☆fn是执行器。compose是合并生成一个执行器的技术，curry是分次加载参数的技术，两者可以共用：
function curryAndcomposeX(fn[]){
	return a=>b=>c=>d => funcs.reduce((prev, current:(arg:any)=>any) => current(prev), [a,b,c,d])
}

将执行器参数和普通参数分开传参的风格，叫做Pointfree style 